In [ ]:
import asyncio
from agents import Agent, Runner, SQLiteSession, function_tool
from agents.extensions.visualization import draw_graph
from pydantic import BaseModel

# 세션 초기화
session = SQLiteSession("user_1", "ai-memory.db")

# Pydantic 모델 정의 (구조화된 출력을 위해)
class Answer(BaseModel):
    answer: str
    background_explanation: str

# 날씨 도구 정의
@function_tool
def get_weather():
    return "30"

# 지리 전문 에이전트 (구조화된 출력과 도구 사용)
geaography_agent = Agent(
    name="Geo Expert Agent",
    instructions="You are a expert in geography, you answer questions related to them.",
    handoff_description="Use this to answer geography related questions.",
    tools=[get_weather],
    output_type=Answer,
)

# 경제 전문 에이전트
economics_agent = Agent(
    name="Economics Expert Agent",
    instructions="You are a expert in economics, you answer questions related to them.",
    handoff_description="Use this to answer economics questions.",
)

# 메인 에이전트 (handoff 관리)
main_agent = Agent(
    name="Main Agent",
    instructions="You are a user facing agent. Transfer to the agent most capable of answering the user's question.",
    handoffs=[
        economics_agent,
        geaography_agent,
    ],
)

async def main():
    # 에이전트 그래프 시각화 (선택사항)
    print("Agent Graph:")
    try:
        graph = draw_graph(main_agent)
        print(f"Graph generated: {type(graph)}")
    except Exception as e:
        print(f"Visualization error: {e}")
    
    print("\n" + "="*50)
    
    # 첫 번째 질문 - 경제학 질문
    print("첫 번째 질문: Why do countries sell bonds?")
    result1 = await Runner.run(
        main_agent,
        "Why do countries sell bonds?",
        session=session,
    )
    
    print(f"처리 에이전트: {result1.last_agent.name}")
    print(f"답변: {result1.final_output}")
    
    print("\n" + "="*50)
    
    # 두 번째 질문 - 지리학 질문 (구조화된 출력)
    print("두 번째 질문: What is the capital of Thailand's northern province?")
    result2 = await Runner.run(
        main_agent,
        "What is the capital of Thailand's northern province?",
        session=session,
    )
    
    print(f"처리 에이전트: {result2.last_agent.name}")
    print(f"답변: {result2.final_output}")

# 비동기 함수 실행
if __name__ == "__main__":
    asyncio.run(main())